In [30]:
#Import required Libraries
import pandas as pd
from datetime import datetime
import openpyxl
import xlwt

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
#Get each dataframe from the respective files in the directory

df_borrowers = pd.read_csv("Borrower_Data.csv")
df_loans = pd.read_csv("Loan_Data.csv")
df_schedules = pd.read_csv("Schedule_Data.xlsx.csv")
df_repayments = pd.read_csv("Repayment_Data.csv")

In [4]:
#Explore the columns of each dataframe
print(list(df_borrowers.columns))
print(list(df_loans.columns))
print(list(df_schedules.columns))
print(list(df_repayments.columns))

['Borrower_id', 'State', 'City', 'zip code', 'borrower_credit_score']
['Borrower_id', 'loan_id', 'Date_of_release', 'Term', 'InterestRate', 'LoanAmount', 'Downpayment', 'Payment_frequency', 'Maturity_date']
['loan_id', 'schedule_id', 'Expected_payment_date', 'Expected_payment_amount']
['loan_id', 'payment_id', 'Amount_paid', 'Date_paid']


In [5]:
#Rename Borrower column 
df_borrowers.rename(columns={'Borrower_Id':'Borrower_id'}, inplace=True)

In [8]:
#Merge Borrowers and Loans tables and inspect the result
#This shows the borrowers and the loans they currently have

borr_loan = df_borrowers.merge(df_loans, on='Borrower_id')
print(borr_loan.shape)
borr_loan.head()

(4, 13)


,Borrower_id,State,City,zip code,borrower_credit_score,loan_id,Date_of_release,Term,InterestRate,LoanAmount,Downpayment,Payment_frequency,Maturity_date
0,123fd35,dfgc,12olki,19473,4,32u09wekjbfje,01/27/2021,12,1.05,1209484,124993,1.007903e+05,01/31/2022
1,123fd35,dfgc,12olki,19473,4,09u924rbwf,04/16/2021,36,1.05,45784937,1200000,1.271804e+06,04/30/2024
2,123fd36,rtgio,89hfg,923475,a,9190i0-nbfb,02/01/2021,24,1.05,12394031,1000000,5.164180e+05,02/29/2023
3,123fd36,rtgio,89hfg,923475,a,3240-9rfwb,03/26/2021,12,1.05,1234890,132000,1.029075e+05,03/31/2022


In [9]:
#Change wrong date value from 02/29/2023 to 02/28/2023 as there is not 29th February in 2023
borr_loan['Maturity_date'] = borr_loan['Maturity_date'].replace(['02/29/2023'], '02/28/2023')

In [10]:
#Concatenate Repayment and Schedules tables and inspect the result
df_repayments.drop(columns=['loan_id'], inplace=True)
repay_sched = pd.concat([df_schedules, df_repayments], axis=1)
repay_sched.shape

(84, 7)

In [39]:
#Merge the final dataframes, which show the payment schedules for each borrower,their respective loans and loan repayments

bl_rs = borr_loan.merge(repay_sched, on='loan_id')
bl_rs.shape

(84, 19)

In [40]:
bl_rs.head()

,Borrower_id,State,City,zip code,borrower_credit_score,loan_id,Date_of_release,Term,InterestRate,LoanAmount,Downpayment,Payment_frequency,Maturity_date,schedule_id,Expected_payment_date,Expected_payment_amount,payment_id,Amount_paid,Date_paid
0,123fd35,dfgc,12olki,19473,4,32u09wekjbfje,01/27/2021,12,1.05,1209484,124993,100790.3333,01/31/2022,3434r409km123456,02/27/2021,100790.3333,3434r409kmPAID123456,100790.3333,03/31/2021
1,123fd35,dfgc,12olki,19473,4,32u09wekjbfje,01/27/2021,12,1.05,1209484,124993,100790.3333,01/31/2022,3434r409km123457,03/27/2021,100790.3333,3434r409kmPAID123457,100790.3333,03/31/2021
2,123fd35,dfgc,12olki,19473,4,32u09wekjbfje,01/27/2021,12,1.05,1209484,124993,100790.3333,01/31/2022,3434r409km123458,04/27/2021,100790.3333,3434r409kmPAID123458,100790.3333,04/27/2021
3,123fd35,dfgc,12olki,19473,4,32u09wekjbfje,01/27/2021,12,1.05,1209484,124993,100790.3333,01/31/2022,3434r409km123459,05/27/2021,100790.3333,3434r409kmPAID123459,100790.3333,05/27/2021
4,123fd35,dfgc,12olki,19473,4,32u09wekjbfje,01/27/2021,12,1.05,1209484,124993,100790.3333,01/31/2022,3434r409km123460,06/27/2021,100790.3333,3434r409kmPAID123460,100790.3333,06/27/2021


In [13]:
#Convert data columns to datetime format to ease transformations

bl_rs['Expected_payment_date']= pd.to_datetime(bl_rs['Expected_payment_date'])
bl_rs['Maturity_date']= pd.to_datetime(bl_rs['Maturity_date'])
bl_rs['Date_paid']= pd.to_datetime(bl_rs['Date_paid'])

In [14]:
#Calculate PAR days, which displays the days missed for each expected payment

bl_rs['par_days'] = abs((bl_rs['Expected_payment_date'] - bl_rs['Date_paid']).dt.days)

In [15]:
bl_rs.head()

,Borrower_id,State,City,zip code,borrower_credit_score,loan_id,Date_of_release,Term,InterestRate,LoanAmount,Downpayment,Payment_frequency,Maturity_date,schedule_id,Expected_payment_date,Expected_payment_amount,payment_id,Amount_paid,Date_paid,par_days
0,123fd35,dfgc,12olki,19473,4,32u09wekjbfje,01/27/2021,12,1.05,1209484,124993,100790.3333,2022-01-31,3434r409km123456,2021-02-27,100790.3333,3434r409kmPAID123456,100790.3333,2021-03-31,32
1,123fd35,dfgc,12olki,19473,4,32u09wekjbfje,01/27/2021,12,1.05,1209484,124993,100790.3333,2022-01-31,3434r409km123457,2021-03-27,100790.3333,3434r409kmPAID123457,100790.3333,2021-03-31,4
2,123fd35,dfgc,12olki,19473,4,32u09wekjbfje,01/27/2021,12,1.05,1209484,124993,100790.3333,2022-01-31,3434r409km123458,2021-04-27,100790.3333,3434r409kmPAID123458,100790.3333,2021-04-27,0
3,123fd35,dfgc,12olki,19473,4,32u09wekjbfje,01/27/2021,12,1.05,1209484,124993,100790.3333,2022-01-31,3434r409km123459,2021-05-27,100790.3333,3434r409kmPAID123459,100790.3333,2021-05-27,0
4,123fd35,dfgc,12olki,19473,4,32u09wekjbfje,01/27/2021,12,1.05,1209484,124993,100790.3333,2022-01-31,3434r409km123460,2021-06-27,100790.3333,3434r409kmPAID123460,100790.3333,2021-06-27,0


In [19]:
#Identify and differentiate months with outstanding debts from months with timely payments

months_outs = []
for par_days in bl_rs["par_days"]:
    if par_days == 0:
        mth = 0
    else:
        mth = 1
    months_outs.append(mth)

#Add resulting series to dataframe 
bl_rs["months_outstanding"] = months_outs

In [20]:
#Calculate amount due based on outstanding months
bl_rs["amount_due"] = (bl_rs["Expected_payment_amount"]) * bl_rs["months_outstanding"]

In [21]:
#Group by borrowers ID, Loans ID and alculate Amount at risk due to delayed payments

bl_rs["amount_at_risk"] = bl_rs.groupby(["Borrower_id", "loan_id", "Date_paid"])["amount_due"].cumsum()

In [23]:
#Get the last due dates and last repayment dates 

bl_rs['last_due_date'] = bl_rs.groupby(["Borrower_id", "Maturity_date", 'loan_id'])['Expected_payment_date'].transform('max')

bl_rs['last_repayment_date'] = bl_rs.groupby(["Borrower_id", "Maturity_date",'loan_id'])['Date_paid'].transform('max')

In [33]:
bl_rs = bl_rs.rename(columns={
    'Borrower_id': 'borrower_id',
    'State': 'state',
    'City': 'city',
    'Payment_frequency': 'payment_frequency',
    'Maturity_date': 'maturity_date',
    'par_days': 'current_days_past_due',
    'last_due_date': 'last_due_date',
    'last_repayment_date': 'last_repayment_date',
    'amount_at_risk': 'amount_at_risk',
 
})

In [35]:
#Select specified columns for final dataframe

df_final = bl_rs[['borrower_id', 'state', 'city', 'zip code', 'payment_frequency', 'maturity_date', 'current_days_past_due', 'last_due_date', 'last_repayment_date', 'amount_at_risk']]

In [36]:
#Inspect final Dataframe

df_final.head()

,borrower_id,state,city,zip code,payment_frequency,maturity_date,current_days_past_due,last_due_date,last_repayment_date,amount_at_risk
0,123fd35,dfgc,12olki,19473,100790.3333,2022-01-31,32,2022-01-27,2022-01-27,100790.3333
1,123fd35,dfgc,12olki,19473,100790.3333,2022-01-31,4,2022-01-27,2022-01-27,201580.6666
2,123fd35,dfgc,12olki,19473,100790.3333,2022-01-31,0,2022-01-27,2022-01-27,0.0000
3,123fd35,dfgc,12olki,19473,100790.3333,2022-01-31,0,2022-01-27,2022-01-27,0.0000
4,123fd35,dfgc,12olki,19473,100790.3333,2022-01-31,0,2022-01-27,2022-01-27,0.0000


In [38]:
#Save Dataframe as Excel file
#Notably, the xls format is no longer supported. However, xlsx is the newly supported format for saving Excel files
#Also xlsx files are smaller in size than xls files

df_final.to_excel("C:/Users/user/Downloads/Loans/output_name.xls")

C:\Users\user\AppData\Local\Temp\ipykernel_428\3074046757.py:4: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  df_final.to_excel("C:/Users/user/Downloads/Loans/output_name.xls")
